In [1]:
import findspark
findspark.init("/opt/spark")

from pyspark import SparkContext
import findspark
import pyspark
import random
import numpy as np
from PIL import Image
from pyspark.sql import SparkSession 
import sys

In [2]:
spark = SparkSession \
    .builder \
    .appName("Img Seg") \
    .master("spark://hadoop-master:7077") \
    .getOrCreate()

22/06/30 05:54:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/30 05:54:46 WARN SparkContext: Please ensure that the number of slots available on your executors is limited by the number of cores to task cpus and not another custom resource. If cores is not the limiting resource then dynamic allocation will not work properly!


In [31]:
# image_path = "./2x2.png"
image_path = "hdfs://hadoop-master:9000/user/hadoopuser/2x2.png"
# image_path = "/home/toa/Pictures/in/"
# result_path = "/home/toa/result.png" 

In [38]:
from PIL import Image
import numpy as np

image_df = spark.read.format("image").load(image_path)

In [39]:
image_df.select("image.height", "image.width"
         ,"image.nChannels", "image.mode"
         ,"image.data").show()

+------+-----+---------+----+--------------------+
|height|width|nChannels|mode|                data|
+------+-----+---------+----+--------------------+
|     2|    2|        3|  16|[4C B1 22 E8 A2 0...|
+------+-----+---------+----+--------------------+



In [45]:
# rdd_img_array = image_df.select("image.data").rdd.flatMap(lambda f: f.select("data"))
rdd_img_array = image_df.select("image.data").rdd.flatMap(lambda f: f.asDict()["data"])
img_data = image_df.select("image.*").rdd.\
    flatMap(lambda row: (row.asDict()["height"],\
                         row.asDict()["width"],\
                         row.asDict()["nChannels"])).collect()

In [52]:
height, width, n_channels = img_data
offset = spark.sparkContext.accumulator("offSetAcc", "int")
x = spark.sparkContext.accumulator("xAcc", "int")
y = spark.sparkContext.accumulator("yAcc", "int")
x.add(1)
y.add(1)

AttributeError: 'str' object has no attribute 'addInPlace'

In [ ]:
from PIL import Image
import numpy as np

image_df = spark.read.format("image").load(image_path)

In [37]:
import pyspark.sql.functions as F
from pyspark.ml.image import ImageSchema
from pyspark.ml.linalg import DenseVector, VectorUDT

img2vec = F.udf(lambda x: DenseVector(ImageSchema.toNDArray(x).flatten()), VectorUDT())

image_df = image_df.withColumn('vecs', img2vec("image"))
image_df.show()

22/06/30 06:15:33 WARN TaskSetManager: Lost task 0.0 in stage 8.0 (TID 26, 25.27.142.242, executor 0): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 605, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 597, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 223, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 141, in dump_stream
    for obj in iterator:
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 212, in _batched
    for item in iterator:
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 450, in mapper
    result = tuple(f(*[a[o] for o in arg_offsets]) for (arg_offsets, f) in udfs)
  File "/opt/spark/python/lib/pyspark.zip/

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 605, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 597, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 223, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 141, in dump_stream
    for obj in iterator:
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 212, in _batched
    for item in iterator:
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 450, in mapper
    result = tuple(f(*[a[o] for o in arg_offsets]) for (arg_offsets, f) in udfs)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 450, in <genexpr>
   

In [19]:
image_df.printSchema()

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)



In [15]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

In [33]:
print(image_df.count(),len(image_df.columns))

1 1


In [27]:
rdd_img = image_df.select("image.data").rdd.flatMap(lambda x: x[0])

In [31]:
img_data = image_df.select("image.*").rdd.map(lambda row: row.asDict()).collect()(0)

22/06/30 05:49:04 WARN TaskSetManager: Lost task 0.0 in stage 6.0 (TID 24, 25.27.142.242, executor 0): java.io.FileNotFoundException: File file:/home/toa/2x2.png does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:124)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:169)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:93)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNe

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 4 times, most recent failure: Lost task 0.3 in stage 6.0 (TID 27, 25.27.142.242, executor 0): java.io.FileNotFoundException: File file:/home/toa/2x2.png does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:124)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:169)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:93)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.sca

In [24]:
vec_assembler = VectorAssembler(inputCols=input_cols, outputCol="features")

In [25]:
final_data = vec_assembler.transform(image_df)

IllegalArgumentException: image.red does not exist. Available: image

In [15]:
kmeans = KMeans(k=3,featuresCol="features")

In [16]:
model = kmeans.fit(final_data)

In [18]:
model

KMeansModel: uid=KMeans_bea20945bdd5, k=3, distanceMeasure=euclidean, numFeatures=4

In [17]:
model.transform(final_data).groupBy("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   50|
|         2|   61|
|         0|   39|
+----------+-----+



In [19]:
predictions = model.transform(final_data)
predictions.show()

+-----------+----------+-----------+----------+-------+-----------------+----------+
|sepalLength|sepalWidth|petalLength|petalWidth|variety|         features|prediction|
+-----------+----------+-----------+----------+-------+-----------------+----------+
|        5.1|       3.5|        1.4|       0.2| Setosa|[5.1,3.5,1.4,0.2]|         1|
|        4.9|       3.0|        1.4|       0.2| Setosa|[4.9,3.0,1.4,0.2]|         1|
|        4.7|       3.2|        1.3|       0.2| Setosa|[4.7,3.2,1.3,0.2]|         1|
|        4.6|       3.1|        1.5|       0.2| Setosa|[4.6,3.1,1.5,0.2]|         1|
|        5.0|       3.6|        1.4|       0.2| Setosa|[5.0,3.6,1.4,0.2]|         1|
|        5.4|       3.9|        1.7|       0.4| Setosa|[5.4,3.9,1.7,0.4]|         1|
|        4.6|       3.4|        1.4|       0.3| Setosa|[4.6,3.4,1.4,0.3]|         1|
|        5.0|       3.4|        1.5|       0.2| Setosa|[5.0,3.4,1.5,0.2]|         1|
|        4.4|       2.9|        1.4|       0.2| Setosa|[4.4,2.9,1

In [21]:
predictions.groupBy("variety","prediction").count().show()

+----------+----------+-----+
|   variety|prediction|count|
+----------+----------+-----+
|Versicolor|         0|    3|
|Versicolor|         2|   47|
| Virginica|         2|   14|
|    Setosa|         1|   50|
| Virginica|         0|   36|
+----------+----------+-----+

